In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
!pip install catboost
!pip install ipywidgets
!jupyter nbextension enable  — py widgetsnbextension

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.6/98.6 MB 2.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 7.8 MB/s eta 0:00:00
Please specify one nbextension/package at a time


In [3]:
df = pd.read_csv("/content/drive/MyDrive/CDSAML_P13/NY.csv", low_memory = False)
df

,Health Service Area,Hospital County,Operating Certificate Number,Facility Id,Facility Name,Age Group,Zip Code - 3 digits,Gender,Race,Ethnicity,...,APR Risk of Mortality,APR Medical Surgical Description,Payment Typology 1,Payment Typology 2,Payment Typology 3,Birth Weight,Abortion Edit Indicator,Emergency Department Indicator,Total Charges,Total Costs
0,Western NY,Allegany,226700.0,37.0,Cuba Memorial Hospital Inc,70 or Older,147,F,White,Not Span/Hispanic,...,Minor,Medical,Medicare,Private Health Insurance,NaN,0,N,Y,5333.90,4818.42
1,Western NY,Allegany,226700.0,37.0,Cuba Memorial Hospital Inc,70 or Older,147,F,White,Not Span/Hispanic,...,Moderate,Medical,Medicare,NaN,NaN,0,N,Y,4865.99,4588.78
2,Western NY,Allegany,226700.0,37.0,Cuba Memorial Hospital Inc,70 or Older,147,F,White,Not Span/Hispanic,...,Minor,Medical,Medicare,Private Health Insurance,NaN,0,N,Y,5901.54,5559.56
3,Western NY,Allegany,226700.0,37.0,Cuba Memorial Hospital Inc,50 to 69,147,F,White,Not Span/Hispanic,...,Minor,Medical,Medicare,NaN,NaN,0,N,Y,3619.08,3567.25
4,Western NY,Allegany,226700.0,37.0,Cuba Memorial Hospital Inc,70 or Older,147,M,White,Not Span/Hispanic,...,Moderate,Medical,Medicare,Blue Cross/Blue Shield,Private Health Insurance,0,N,Y,3185.87,3167.89
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2346926,Long Island,Suffolk,5157003.0,943.0,St Catherine of Siena Hospital,70 or Older,117,M,White,Not Span/Hispanic,...,Moderate,Medical,Medicare,Medicare,Private Health Insurance,0,N,Y,43494.00,5588.26
2346927,Long Island,Suffolk,5157003.0,943.0,St Catherine of Siena Hospital,50 to 69,117,M,White,Not Span/Hispanic,...,Minor,Medical,Private Health Insurance,NaN,NaN,0,N,Y,16662.00,2907.62
2346928,NaN,NaN,NaN,NaN,Redacted for Confidentiality,50 to 69,NaN,M,White,Unknown,...,Minor,Medical,Medicaid,NaN,NaN,0,N,N,7142.00,1692.55
2346929,Long Island,Suffolk,5157003.0,943.0,St Catherine of Siena Hospital,70 or Older,117,M,White,Not Span/Hispanic,...,Moderate,Medical,Medicare,Medicare,Blue Cross/Blue Shield,0,N,Y,13070.00,2447.75


In [4]:
df = df.drop(["Facility Id", "Total Charges", "Total Costs", "Health Service Area", "Hospital County","Zip Code - 3 digits", "Race", "Ethnicity", "Patient Disposition", "Birth Weight", "Payment Typology 3", "Payment Typology 2", "Operating Certificate Number","Facility Name", "Gender", "CCS Diagnosis Description", "CCS Procedure Description", "APR DRG Description", "APR MDC Description", "APR Severity of Illness Description", "APR Medical Surgical Description", "Abortion Edit Indicator", "Discharge Year"], axis = 1)

In [5]:
df = df.dropna(subset=['APR Risk of Mortality'])

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2346820 entries, 0 to 2346930
Data columns (total 11 columns):
 #   Column                          Dtype 
---  ------                          ----- 
 0   Age Group                       object
 1   Length of Stay                  object
 2   Type of Admission               object
 3   CCS Diagnosis Code              int64 
 4   CCS Procedure Code              int64 
 5   APR DRG Code                    int64 
 6   APR MDC Code                    int64 
 7   APR Severity of Illness Code    int64 
 8   APR Risk of Mortality           object
 9   Payment Typology 1              object
 10  Emergency Department Indicator  object
dtypes: int64(5), object(6)
memory usage: 214.9+ MB


In [7]:
df['Length of Stay'] = df['Length of Stay'].apply(lambda x: str(x).split(' ')[0])
df['Length of Stay'] = pd.to_numeric(df['Length of Stay'])

In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2346820 entries, 0 to 2346930
Data columns (total 11 columns):
 #   Column                          Dtype 
---  ------                          ----- 
 0   Age Group                       object
 1   Length of Stay                  int64 
 2   Type of Admission               object
 3   CCS Diagnosis Code              int64 
 4   CCS Procedure Code              int64 
 5   APR DRG Code                    int64 
 6   APR MDC Code                    int64 
 7   APR Severity of Illness Code    int64 
 8   APR Risk of Mortality           object
 9   Payment Typology 1              object
 10  Emergency Department Indicator  object
dtypes: int64(6), object(5)
memory usage: 214.9+ MB


# Two Bins

In [9]:
bins = [0,6,120]
labels = [6,120]
df['stay_bin'] = pd.cut(x = df['Length of Stay'], bins = bins)
df['stay_label'] = pd.cut(x = df['Length of Stay'], bins = bins, labels = labels)
df['stay_bin'] = df['stay_bin'].apply(lambda x: str(x).replace(',',' -'))
df['stay_bin'] = df['stay_bin'].apply(lambda x: str(x).replace('120','120+'))
df

,Age Group,Length of Stay,Type of Admission,CCS Diagnosis Code,CCS Procedure Code,APR DRG Code,APR MDC Code,APR Severity of Illness Code,APR Risk of Mortality,Payment Typology 1,Emergency Department Indicator,stay_bin,stay_label
0,70 or Older,4,Urgent,122,0,139,4,2,Minor,Medicare,Y,(0 - 6],6
1,70 or Older,4,Elective,55,0,422,10,2,Moderate,Medicare,Y,(0 - 6],6
2,70 or Older,4,Urgent,122,202,139,4,1,Minor,Medicare,Y,(0 - 6],6
3,50 to 69,2,Elective,55,0,249,6,2,Minor,Medicare,Y,(0 - 6],6
4,70 or Older,2,Elective,122,0,139,4,1,Moderate,Medicare,Y,(0 - 6],6
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2346926,70 or Older,2,Emergency,112,0,47,1,2,Moderate,Medicare,Y,(0 - 6],6
2346927,50 to 69,1,Emergency,54,159,351,8,1,Minor,Private Health Insurance,Y,(0 - 6],6
2346928,50 to 69,2,Urgent,657,0,751,19,1,Minor,Medicaid,N,(0 - 6],6
2346929,70 or Older,1,Emergency,106,0,201,5,1,Moderate,Medicare,Y,(0 - 6],6


In [10]:
from catboost import CatBoostClassifier, Pool
from sklearn.model_selection import train_test_split

cb = CatBoostClassifier(n_estimators=200,
                       loss_function='MultiClass',
                       learning_rate=0.1,
                       depth=6,
                       random_state=1,
                       verbose=False)

new_X = df.drop(['Length of Stay','stay_bin','stay_label'], axis=1)
new_y = df[['stay_label']]
X_train, X_test, y_train, y_test = train_test_split(new_X, new_y, test_size=0.30, random_state=1)

pool_train = Pool(X_train, y_train,
                  cat_features = ['Age Group', 'Type of Admission', 'APR Risk of Mortality', 'Payment Typology 1', 'Emergency Department Indicator'])

pool_test = Pool(X_test, cat_features = ['Age Group', 'Type of Admission', 'APR Risk of Mortality', 'Payment Typology 1', 'Emergency Department Indicator'])

cb.fit(pool_train)
y_pred = cb.predict(pool_test)

import numpy as np
from sklearn.metrics import accuracy_score

cb_accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", cb_accuracy)

Accuracy: 0.8504742587842272


In [11]:
from sklearn.metrics import mean_squared_error
preds = cb.predict(pool_test)
rmse = mean_squared_error(y_test, preds, squared=False)
print(f"RMSE of CB model with 2 bins: {rmse:.3f}")

RMSE of CB model with 2 bins: 44.082


# Many Bins

In [12]:
bins = [0,6,12,30,120]
labels = [6,12,30,120]
df['stay_bin'] = pd.cut(x = df['Length of Stay'], bins = bins)
df['stay_label'] = pd.cut(x = df['Length of Stay'], bins = bins, labels = labels)
df['stay_bin'] = df['stay_bin'].apply(lambda x: str(x).replace(',',' -'))
df['stay_bin'] = df['stay_bin'].apply(lambda x: str(x).replace('120','120+'))
df

,Age Group,Length of Stay,Type of Admission,CCS Diagnosis Code,CCS Procedure Code,APR DRG Code,APR MDC Code,APR Severity of Illness Code,APR Risk of Mortality,Payment Typology 1,Emergency Department Indicator,stay_bin,stay_label
0,70 or Older,4,Urgent,122,0,139,4,2,Minor,Medicare,Y,(0 - 6],6
1,70 or Older,4,Elective,55,0,422,10,2,Moderate,Medicare,Y,(0 - 6],6
2,70 or Older,4,Urgent,122,202,139,4,1,Minor,Medicare,Y,(0 - 6],6
3,50 to 69,2,Elective,55,0,249,6,2,Minor,Medicare,Y,(0 - 6],6
4,70 or Older,2,Elective,122,0,139,4,1,Moderate,Medicare,Y,(0 - 6],6
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2346926,70 or Older,2,Emergency,112,0,47,1,2,Moderate,Medicare,Y,(0 - 6],6
2346927,50 to 69,1,Emergency,54,159,351,8,1,Minor,Private Health Insurance,Y,(0 - 6],6
2346928,50 to 69,2,Urgent,657,0,751,19,1,Minor,Medicaid,N,(0 - 6],6
2346929,70 or Older,1,Emergency,106,0,201,5,1,Moderate,Medicare,Y,(0 - 6],6


In [14]:
from catboost import CatBoostClassifier, Pool
from sklearn.model_selection import train_test_split

cb = CatBoostClassifier(n_estimators=200,
                       loss_function='MultiClass',
                       learning_rate=0.1,
                       depth=6,
                       random_state=1,
                       verbose=False)

new_X = df.drop(['Length of Stay','stay_bin','stay_label'], axis=1)
new_y = df[['stay_label']]
X_train, X_test, y_train, y_test = train_test_split(new_X, new_y, test_size=0.30, random_state=1)

pool_train = Pool(X_train, y_train,
                  cat_features = ['Age Group', 'Type of Admission', 'APR Risk of Mortality', 'Payment Typology 1', 'Emergency Department Indicator'])

pool_test = Pool(X_test, cat_features = ['Age Group', 'Type of Admission', 'APR Risk of Mortality', 'Payment Typology 1', 'Emergency Department Indicator'])

cb.fit(pool_train)
y_pred = cb.predict(pool_test)

import numpy as np
from sklearn.metrics import accuracy_score

cb_accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", cb_accuracy)

Accuracy: 0.8040057041727388


In [15]:
from sklearn.metrics import mean_squared_error
preds = cb.predict(pool_test)
rmse = mean_squared_error(y_test, preds, squared=False)
print(f"RMSE of CB model with many bins: {rmse:.3f}")

RMSE of CB model with many bins: 13.581


# Normalized Dataset

In [ ]:
import numpy as np
from catboost import CatBoostClassifier, Pool
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score

cb = CatBoostClassifier(n_estimators=100,
                       loss_function='MultiClass',
                       learning_rate=0.1,
                       depth=6,
                       random_state=1,
                       verbose=False)

new_X = df.drop(['Length of Stay','stay_bin','stay_label'], axis=1)
new_y = df[['stay_label']]
X_train, X_test, y_train, y_test = train_test_split(new_X, new_y, test_size=0.30, random_state=1)

numeric_columns = ['CCS Diagnosis Code', 'CCS Procedure Code', 'APR DRG Code', 'APR MDC Code', 'APR Severity of Illness Code']
categoric_columns = ['Age Group', 'Type of Admission', 'APR Risk of Mortality', 'Payment Typology 1', 'Emergency Department Indicator']

X_train[numeric_columns] = StandardScaler().fit_transform(X_train[numeric_columns])
print(X_train)
X_test[numeric_columns] = StandardScaler().fit_transform(X_test[numeric_columns])
print(X_test)

pool_train = Pool(X_train, y_train, cat_features = categoric_columns)
pool_test = Pool(X_test, cat_features = categoric_columns)

cb.fit(pool_train)
y_pred = cb.predict(pool_test)

cb_accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", cb_accuracy)